# Create New Tenant

This chains together a lot of concepts from other notebooks.

In [ ]:
pip install azure-mgmt-sql requests msal pyodbc requests_toolbelt

In [ ]:
#For orgaization purposes I put notbooks in subfolders not the root of the proejct.aad_token
#This code adds the root directory of the project to the sys path so we can load class modules from the services folder
#I think this only needs to be run once, but including it for completeness.
import os, sys
projectRoot = os.path.abspath('.')
directory = os.path.dirname(projectRoot)
if not directory in sys.path: sys.path.append(directory)

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login
#Here we're just getting the credential with out a scope because we'll request different scopes throughout this example
from services.aadservice import AadService
credential = AadService.get_credential()

## Part 1 - Find Next Tenant Name


In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login
#Retrieving for the Power BI scope here 
scope = 'https://analysis.windows.net/powerbi/api/.default'
aadPBIToken = credential.get_token(scope).token

pbiApiHeaders =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + aadPBIToken}

In [ ]:
#Here we're retreiving a list of power bi workspaces
import requests
import json

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups'       

apiResponse = requests.get(apiUrl, headers=pbiApiHeaders)
#error handling for API call
if apiResponse.status_code != 200:
    description = f'Error retreiving workspace:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    print(json.dumps(apiResponse,indent=2))

In [ ]:
#In this cell we're iterating the list of workspaces to find the max suffix for our "tenantRoot" value defined in our constnats.
#This helps us name the tenants resources consistently with an ordinal suffix.
from services.env import const

workspaces = apiResponse["value"]

keyPosition = 0
keyValue = 0

for workspace in workspaces:
    nameParts = workspace["name"].split(const.tenantRoot)
    #if there was an underscroe in the title, get the suffix of the database name.
    if len(nameParts) > 1:
        #if the suffix is greater than our max capture it as the new max
        if int(nameParts[1]) > keyValue:
            keyValue = int(nameParts[1])
keyValue = keyValue+1
tenantName = f"{const.tenantRoot}{keyValue}"

print(f"Next tenant is: {tenantName}")

## Part 2 - Create Tenant Workspace

In [ ]:
#This creates the new pwoer bi workspace with the generated tenant name.  It's important for us to grab the workspace id at this time
#we'll need it later.
import requests
import json
from services.env import const

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups?workspaceV2=True'       

body = {"name":tenantName}

apiResponse = requests.post(apiUrl, headers=pbiApiHeaders, data=json.dumps(body))
#error handling for create workspace
if apiResponse.status_code != 200:
    description = f'Error creating workspace:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    workspace = json.loads(apiResponse.text)
    workspaceId = workspace["id"]
    print(f"Workspace {workspaceId} created")

In [ ]:
#By default the API caller will be the only user with permissions on the workspace.  Here we add our global admin user from
#the constants definition so we can login to powerbi.com and inspect the workspace.
import requests
import json
from services.env import const

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/users'       

body = {
  "emailAddress": const.globalAdminUser,
  "groupUserAccessRight": "Admin"
}

apiResponse = requests.post(apiUrl, headers=pbiApiHeaders, data=json.dumps(body))
#error handling for adding user
if apiResponse.status_code != 200:
    description = f'Error creating user:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    print(f"User {const.globalAdminUser} added")

## Part 3:  Create Tenant Capacity

In [ ]:
#Here we're getting a token for the azure management scope so we can create azure resources.
scope = 'https://management.azure.com/.default'
azureMgmtToken = credential.get_token(scope).token

azureApiHeaders =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + azureMgmtToken}

In [ ]:
#And now creating the fabric capacity with the prescribed tenant name, and the list of administrators from the constants file.
#The administrators list is important, because only administrators will be able to see the capacity in the Power BI API.
import requests
import json
from services.env import const

apiUrl = f'https://management.azure.com/subscriptions/{const.subscriptionId}/resourceGroups/{const.resourceGroup}/providers/Microsoft.Fabric/capacities/{tenantName}?api-version=2022-07-01-preview'       

body = { 
            "type": "Microsoft.Fabric/capacities",
            "name": tenantName,
            "location": "westus3",
            "sku": {
                "name": "F2",
                "tier": "Fabric"
            },
            "properties": {
                "administration": {
                    "members": const.administrators
                }
            }
        }

apiResponse = requests.put(apiUrl, headers=azureApiHeaders, data=json.dumps(body))

#error handling for create capacity
if apiResponse.status_code != 201 and apiResponse.status_code != 200:
    description = f'Error creating capacity:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    print(f"Capacity '{tenantName}' created")

## Part 4:  Add Capacity to Workspace

In [ ]:
#Now we're looking for the capacity we just created in the Power BI API, matching it by display name to get its id.
import requests
import json

capacityId = "blank"
apiUrl = f'https://api.powerbi.com/v1.0/myorg/capacities'      

apiResponse = requests.get(apiUrl, headers=pbiApiHeaders)
#error handling for createTemporaryUplodadLocation
if apiResponse.status_code != 200:
    description = f'Error creating workspace:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    capacities = apiResponse["value"]
    for capacity in capacities:
        if capacity["displayName"] == tenantName:
            capacityId = capacity["id"]

    print(f"The capacity {tenantName} has id {capacityId}")

In [ ]:
#Now that we know the capacity id and the workspace id, we can assign the capacity to the workspace.
import requests
import json

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/AssignToCapacity'      

body = {'capacityId': capacityId}

apiResponse = requests.post(apiUrl, headers=pbiApiHeaders, data=json.dumps(body))

#error handling for capacity assignment.
if apiResponse.status_code != 200:
    description = f'Error assigning capacity:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    print(f"Capacity {capacityId} assigned to workspace {workspaceId}")

## Part 5: Create Tenant Service Principal

In [ ]:
#Now we get a token for the graph api because we need to work with entra users.
scope = 'https://graph.microsoft.com/.default'

#with the credential object, get the token for the azure management scope.
graphApiToken = credential.get_token(scope).token
graphApiHeaders =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + graphApiToken}

In [ ]:
#First we create a new application specific to this tenant.  This is a security best practice.
import requests
import json

#appId = str(uuid.uuid4())
apiUrl = f'https://graph.microsoft.com/v1.0/applications'       

body = {
    "displayName": tenantName,
    "passwordCredentials":[
        {
            "displayName": "auth secret"
        }
    ]
}

apiResponse = requests.post(apiUrl, headers=graphApiHeaders, data=json.dumps(body))

#error handling for create capacity
if apiResponse.status_code != 201 and apiResponse.status_code != 200:
    description = f'Error creating capacity:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    print(json.dumps(apiResponse,indent=2))

In [ ]:
#Since we creted an application with an auth secret, we need to persist that secret in keyvault so we can use it later.
from services.secretservice import SecretService
from services.env import const

#Grab the app id and client secret for the created application.
appId = apiResponse["appId"]
secrets = apiResponse["passwordCredentials"]
secret = secrets[0]
secretText = secret["secretText"]

#SecretService.get_secret_byname(keyVault,"foo")
appIdKey = f'{tenantName}Id'
appSecretKey = f'{tenantName}Secret'

SecretService.store_secret_byname(const.keyVault, appIdKey, appId)
SecretService.store_secret_byname(const.keyVault, appSecretKey, secretText)

In [ ]:
#Now that the application is created, we need to convert it to a service principal.
import requests
import json

#appId = str(uuid.uuid4())
apiUrl = f'https://graph.microsoft.com/v1.0/servicePrincipals'       

body = {
  "appId": appId
}

apiResponse = requests.post(apiUrl, headers=graphApiHeaders, data=json.dumps(body))

#error handling for create capacity
if apiResponse.status_code != 201 and apiResponse.status_code != 200:
    description = f'Error creating capacity:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    print(f"Application {appId} converted to Service Principal")

## Part 6:  Create Azure SQL Database

Extensive details on what this script does are covered in CreateSQLDB.ipynb.  In this step we're creating a new database as a copy of an existing "template" database.   While there are multiple ways to create an empty database with a specific schema this is the fewest steps.  Other options include:

1. Create a blank database and deploy a schema from a script.
1. Create the database as a restore from a backup file.

Another advantage of this approach is if there are any users that need to have admin access to all databases this user grant, or static data that needs to be included, it can all exist in the database already and will be there already after the copy.

In [ ]:
#In our example we have a blank "template" database that has the schema defined but is essentially empty.
#Here we create a new database for the tenant as a clone of the existing template databsae.
from azure.mgmt.sql import SqlManagementClient
import json
from services.env import const

try:
    #
    sqlClient = SqlManagementClient(credential=credential, subscription_id=const.subscriptionId)
    
    # Create database
    database = sqlClient.databases.begin_create_or_update(
        const.resourceGroup,
        const.serverName,
        tenantName,
        {
            "location": const.location,
            "sku": {
                "name": "S0",
                "tier": "Standard"
            },
            "properties": {
                "createMode": "Copy",
                "sourceDatabaseId": f"/subscriptions/{const.subscriptionId}/resourceGroups/{const.resourceGroup}/providers/Microsoft.Sql/servers/{const.serverName}/databases/template"
            }
        }
        ).result()

    print(f"Database {tenantName} created")
except KeyError:
    print(f"Database {tenantName} create failed")


Now that we've created a database on the precreated server, we want to do some post configuration on that database.  The server's admin account is running as a service principal.  In order to let us connect to that datbabase directly we need to add an additional Entra user to the database.  I'll do this using and ODBC connection and a couple SQL commands to create a local database user that corresponds to an Entra user and also grant that user owner permissions on the database.

This is an example of where you'd likely create a tenant specific user for the database and grant that user access.

Supporting docs:

1. How to install SQL ODBC drivers on Linux:  https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline
2. How to connect to a database with pyodbc drivers and a service principal login:  https://learn.microsoft.com/en-us/azure/azure-sql/database/azure-sql-passwordless-migration-python?view=azuresql&tabs=sign-in-azure-cli%2Cazure-portal-create%2Cazure-portal-assign%2Capp-service-identity

In [ ]:
#Here we connect to the sql database with the primary service principal, and exectue some commands to grant the app service principal
#reader access.
import struct
import pyodbc
import json
from services.secretservice import SecretService
from services.env import const

serverFqdn = f'{const.serverName}.database.windows.net'
driver = '{ODBC Driver 18 for SQL Server}'

#Get a credential for database access.
tokenBytes = credential.get_token("https://database.windows.net/.default").token.encode("UTF-16-LE")
token_struct = struct.pack(f'<I{len(tokenBytes)}s', len(tokenBytes), tokenBytes)
SQL_COPT_SS_ACCESS_TOKEN = 1256

#open the connection
conn_str = f'DRIVER={driver};SERVER={serverFqdn};DATABASE={tenantName};'
conn = pyodbc.connect(conn_str,attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct})

#Update our "WhoAmI" table so we have a basic table to prove our reports are connected to another database
sql = f"TRUNCATE TABLE WhoAmI\n\rINSERT INTO WhoAmI VALUES ('{tenantName}')"
conn.execute(sql)
sql = f"CREATE USER [{tenantName}] FROM EXTERNAL PROVIDER;"
conn.execute(sql)
sql = f"ALTER ROLE db_datareader ADD MEMBER [{tenantName}];"
conn.execute(sql)

#This is important, the connection doesn't auto commit
conn.commit()

print(f"Application {appId} granted data reader permissions on database {tenantName}")

## Part 7 - Deploy Power BI Model to workspace

In [ ]:
#For our template database we have a prebuilt pbix model that has parameters for servername and database name.
#Here we deploy this pbix file to the new tenant workspace
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder

fileLocation = '../files/Template.pbix'

url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/imports?datasetDisplayName={tenantName}'
headers = {
    'Content-Type': 'multipart/form-data',
    'authorization': 'Bearer ' + aadPBIToken
}

# you need this dictionary to convert a binary file into form-data format
# None here means we skip the filename and file content is important 
files = {'value': (None, open(fileLocation, 'rb'), 'multipart/form-data')}

mp_encoder = MultipartEncoder(fields=files)

r = requests.post(
    url=url,
    data=mp_encoder,  # The MultipartEncoder is posted as data, don't use files=...!
    # The MultipartEncoder provides the content-type header with the boundary:
    headers=headers
)

print(f"File {fileLocation} deployed to workspace {workspaceId}")

## Part 8 - Change Dataset Connection

In [ ]:
#/myorg/groups/{workspace_id}/datasets
#This retreives a list of datasets in the given workspace, notice the response value is wrapped in an array []
import requests
import json
import time

#Injecting a wait to make sure Power BI has had time to enumerate datasets in the uploaded model.
time.sleep(15)

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets'       


# Generate Embed token for multiple workspaces, datasets, and reports. Refer https://aka.ms/MultiResourceEmbedToken
apiResponse = requests.get(apiUrl, headers=pbiApiHeaders)
if apiResponse.status_code != 200:
    description = f'Error while retrieving datasets:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
    #print(api_response.status_code, description=description)
else:
    apiResponse = json.loads(apiResponse.text)
    datasets = apiResponse["value"]
    dataset = datasets[0]
    datasetId = dataset["id"]

    print(f"Deployed model has datasetId: {datasetId}")

In [ ]:
#Here we're setting values for the dataset paremeters so on refres the model connects to the correct database.
import requests
import json

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets/{datasetId}/Default.UpdateParameters'       

body = {
    "updateDetails": [
        {
            "name": "server",
            "newValue": serverFqdn
        },
        {
            "name": "database",
            "newValue": tenantName
        }
    ]
}

apiResponse = requests.post(apiUrl, headers=pbiApiHeaders, data=json.dumps(body))
if apiResponse.status_code != 200:
    description = f'Error while updating dataset parameters:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
    #print(api_response.status_code, description=description)
else:
    print(f"Dataset {datasetId} parameters changed successfully")


In [ ]:
import requests
import json
apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets/{datasetId}/datasources'       

apiResponse = requests.get(apiUrl, headers=pbiApiHeaders)
if apiResponse.status_code != 200:
    description = f'Error while retrieving datasets:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
    #print(api_response.status_code, description=description)
else:
    apiResponse = json.loads(apiResponse.text)
    datasources = apiResponse["value"]
    datasource = datasources[0]
    datasourceId = datasource["datasourceId"]
    gatewayId = datasource["gatewayId"]
    print(f"Dataset {datasetId} has datasource {datasourceId} on gateway {gatewayId}")

In [ ]:
#Here we're getting a sql auth token and setting the credentials on the datasource.
#
#Note:  
#The API doesn't presently support passing an oAuth2 refresh token.  As such this token will be short lived (1 hour).
#in order to overcome this limitation the only present option with the API is using sql authentication.
#this will be addressed in the future with an updated API in fabric.
import requests
import json
from services.aadservice import AadService

apiUrl = f'https://api.powerbi.com/v1.0/myorg/gateways/{gatewayId}/datasources/{datasourceId}'       

#We need a SQL Auth token for the new tenant login we created
credential = AadService.get_tenant_credential(tenantName)
aadSQLToken = credential.get_token("https://database.windows.net/.default").token

serialized_credentials = '{\'credentialData\':[{\'name\':\'accessToken\',\'value\':\'' + aadSQLToken + '\'}]}'

body = {
    "credentialDetails": {
        "credentialType": "OAuth2",
        "credentials": serialized_credentials,
        "encryptedConnection": "NotEncrypted",
        "encryptionAlgorithm": "None",
        "privacyLevel": "None"
    }
}


apiResponse = requests.patch(apiUrl, data=json.dumps(body), headers=pbiApiHeaders)
if apiResponse.status_code != 200:
    description = f'Error while retrieving datasets:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
    #print(api_response.status_code, description=description)
else:
    print(f"Updated credentials on datasource {datasourceId}")


## Part 9 - Refresh Model

In [ ]:
#Finally now that we have an auth token applied to the connection, we can kick off a model refresh
import requests
import json

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets/{datasetId}/refreshes'       

body = {
  "notifyOption": "NoNotification",
  "retryCount": 3
}

apiResponse = requests.post(apiUrl, data=json.dumps(body), headers=pbiApiHeaders)
if apiResponse.status_code != 202:
    description = f'Error while retrieving datasets:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
    #print(api_response.status_code, description=description)
else:
    print(f"Refresh on dataset {datasetId} in workspace {workspaceId} initiated Successfully")